# Moodle initial scraper

This is the initial scraper which pulls all your grades for each competency and stores them within `MoodleGrades.csv`. Once you have done this initially you only need to check those competencies which have not yet been graded. Therefore, only use this notebook once, and from then onwards use the notebook found at http://simonbiggs.net/moodlescraper_remaining.

## Interface description

This is a [jupyter notebook](http://jupyter.org). You are likely initially viewing this as a static webpage. To actually run the code you need to [download this notebook](https://raw.githubusercontent.com/SimonBiggs/TEAP-Resources/master/moodle_resources/Moodle%20initial%20scraper.ipynb), and then to run it you need to install [Anaconda3 Python](https://www.continuum.io/downloads#_windows).

Once you are running this notebook on your local machine code can be run by clicking the relevant cell and then pressing `Shift + Enter`. This notebook has been designed so that the cells are to be run one after the other. You should observe the output of each cell before continuing onto the next cell.


In [13]:
import requests
import re
import getpass

import numpy as np
import pandas as pd

In [2]:
session = requests.session()

In [3]:
username = input("username: ")
password = getpass.getpass("password: ")

payload = {
    "loginusername": username,
    "loginpassword": password
}

acpsemloginurl = "https://www.acpsem.org.au/member-centre"

session.post(
    acpsemloginurl,
    data = payload,
    headers = dict(referer=acpsemloginurl)
)

del password
del payload

username: mail@simonbiggs.net
password: ········


In [4]:
moodleloginurl = "http://acpsem.moodle.com.au/auth/association_online/force_login.php"

session.get(moodleloginurl)

<Response [200]>

In [5]:
submission_status_string = (
    '<td class="cell c0" style="">Submission status</td>\\n'
    '<td class="(submissionstatussubmitted )?cell c1 lastcol" style="">(.*)</td>')
grading_status_string = (
    '<td class="cell c0" style="">Grading status</td>\\n'
    '<td class="submission(not)?graded cell c1 lastcol" style="">(.*)</td>')
grade_string = (
    '<td class="cell c0" style="">Grade</td>\n'
    '<td class="cell c1 lastcol" style="">(.*)&nbsp;/&nbsp;1.00</td>'
)

In [6]:
moodle_id_list = [
    3143, 3145, 3144, 3164, 3146, 3148, 3147, 3149, 3150, 
    3151, 3152, 1494, 1495, 1496, 1497, 1498, 1499, 1500, 
    1501, 1502, 1503, 1504, 1505, 1506, 1507, 1508, 1509, 
    1510, 1511, 1512, 1513, 1514, 1515, 1516, 1517, 1518, 
    1519, 1520, 1521, 1522, 1523, 1524, 1525, 1526, 1527, 
    1528, 1529, 1530, 1626, 1627, 1628, 1629, 1633, 1634, 
    1635, 1636, 1637, 1638, 1639, 1640, 1641, 1642, 1643, 
    1644, 1645, 1646, 1647, 1648, 1649, 1650, 1651, 1652, 
    1653, 1654, 1655, 1657, 1658, 2254, 2255, 2256, 2257, 
    2258, 2259, 2260, 2262, 2263, 2266, 2267, 2274, 2275, 
    2276, 2277, 2278, 2279, 2280, 2281, 2282, 2292, 2293, 
    2294, 2295, 2296, 2297, 2298, 2299, 2300, 2301, 2302, 
    2306, 2307, 2308, 2309, 2622, 2628, 2629, 2630, 2631, 
    2632, 2633, 2634, 2635, 2636, 2637, 2638, 2639, 2640, 
    2641, 2642, 2664, 2665, 2666, 2667, 2668, 2669, 2670, 
    2671, 2672, 2673, 2674, 2675, 2676, 2677, 2678, 2679, 
    2680, 2681, 2682, 2683, 2684, 2685, 2995, 3025, 3026, 
    3027, 3028, 3029, 3030, 3031, 3032, 3033, 3034, 3035, 
    3036, 3037, 3038, 3039, 3040, 3041, 3042, 3044, 3045, 
    3046, 3047, 3048, 3049, 3050, 3051, 3052, 3053, 3054, 
    3055, 3056, 3057, 3058, 3059, 3060, 3061, 3022, 3113, 
    3114, 3115, 3116, 3117, 3118, 3121, 3119, 3120, 3322, 
    3323, 3325, 3326, 3327, 3330, 3331, 3332, 3335, 3336, 
    3337, 3340, 3341, 3345, 3348, 3349, 3350, 3353, 3354, 
    3355
]

ctg_id_list = [
    "1.1.1.1", "1.1.1.2", "1.1.1.3", "1.1.1.4", "1.1.2.1", "1.2.1.1", 
    "1.2.1.2", "1.2.1.3", "1.2.1.4", "1.2.2.1", "1.3.1.1", "2.1.1.1", 
    "2.1.1.2", "2.1.1.3", "2.1.1.4", "2.1.2.1", "2.1.2.2", "2.1.3.1", 
    "2.2.1.1", "2.2.1.2", "2.2.2.1", "2.2.2.2", "2.2.2.3", "2.2.3.1", 
    "2.3.1.1", "2.3.1.2", "2.3.1.3", "2.3.2.1", "2.3.2.2", "2.3.2.3", 
    "2.3.3.1", "2.4.1.1", "2.4.1.2", "2.4.1.3", "2.4.1.4", "2.4.2.1", 
    "2.4.2.2", "2.4.3.1", "2.5.1.1", "2.5.2.1", "2.5.2.2", "2.5.2.3", 
    "2.5.3.1", "2.6.1.1", "2.6.1.2", "2.6.2.1", "2.6.2.2", "2.6.3.1", 
    "3.1.1.1", "3.1.1.2", "3.1.1.3", "3.1.1.4", "3.1.2.1", "3.1.2.2", 
    "3.1.3.1", "3.2.1.1", "3.2.2.1", "3.2.2.2", "3.2.3.1", "3.3.1.1", 
    "3.3.1.2", "3.3.1.3", "3.3.2.1", "3.3.2.2", "3.3.2.3", "3.3.2.4", 
    "3.3.3.1", "3.4.1.1", "3.4.1.2", "3.4.2.1", "3.4.2.2", "3.4.3.1", 
    "3.5.1.1", "3.5.1.2", "3.5.2.1", "3.5.2.2", "3.5.3.1", "4.1.1.1", 
    "4.1.1.2", "4.1.2.1", "4.1.2.2", "4.1.2.3", "4.1.3.1", "4.2.1.1", 
    "4.2.1.2", "4.2.2.1", "4.2.2.2", "4.2.3.1", "4.3.1.1", "4.3.2.1", 
    "4.3.3.1", "4.4.1.1", "4.4.1.2", "4.4.1.3", "4.4.2.1", "4.4.2.2", 
    "4.4.3.1", "4.5.1.1", "4.5.2.1", "4.5.2.2", "4.5.3.1", "4.6.1.1", 
    "4.6.1.2", "4.6.2.1", "4.6.2.2", "4.6.2.3", "4.6.3.1", "4.7.1.1", 
    "4.7.2.1", "4.7.2.2", "4.7.2.3", "4.7.3.1", "5.1.1.1", "5.1.1.2", 
    "5.1.1.3", "5.1.1.4", "5.1.1.5", "5.1.2.1", "5.1.2.2", "5.1.2.3", 
    "5.1.2.4", "5.1.2.5", "5.1.3.1", "5.2.1.1", "5.2.1.2", "5.2.2.1", 
    "5.2.2.2", "5.2.3.1", "5.3.1.1", "5.3.1.2", "5.3.1.3", "5.3.1.4", 
    "5.3.1.5", "5.3.1.6", "5.3.2.1", "5.3.2.2", "5.3.2.3", "5.3.3.1", 
    "5.4.1.1", "5.4.1.2", "5.4.2.1", "5.4.3.1", "5.5.1.1", "5.5.1.2", 
    "5.5.1.3", "5.5.2.1", "5.5.2.2", "5.5.2.3", "5.5.2.4", "5.5.3.1", 
    "6.1.1.1", "6.1.1.2", "6.1.2.1", "6.1.2.2", "6.2.1.1", "6.2.1.2", 
    "6.2.1.3", "6.2.2.1", "6.2.2.2", "6.3.1.1", "6.3.1.2", "6.3.2.1", 
    "6.4.1.1", "6.4.2.1", "6.4.2.2", "6.5.1.1", "6.5.1.2", "6.5.1.3", 
    "6.5.1.4", "6.5.2.1", "6.5.2.2", "6.5.2.3", "6.5.2.4", "6.5.2.5", 
    "6.6.1.1", "6.6.1.2", "6.6.1.3", "6.6.1.4", "6.6.2.1", "6.6.2.2", 
    "6.6.2.3", "6.6.2.4", "6.7.3.1", "6.7.3.2", "6.7.3.3", "6.7.3.4", 
    "6.7.3.5", "7.1.1.1", "7.1.1.2", "7.1.1.3", "7.2.1.1", "7.2.1.2", 
    "7.2.1.3", "7.2.2.1", "7.3.1.1", "7.4.1.1", "7.4.2.1", "8.1.1.1", 
    "8.1.1.2", "8.2.1.1", "8.2.1.2", "8.2.1.3", "8.3.1.1", "8.3.1.2", 
    "8.3.1.3", "8.4.1.1", "8.4.1.2", "8.4.1.3", "8.5.1.1", "8.5.1.2", 
    "8.5.1.3", "8.6.1.1", "8.6.1.2", "8.6.1.3", "8.7.1.1", "8.7.1.2", 
    "8.7.1.3"
]

In [7]:
submission_status = []
grading_status = []
grade = []

for i, moodle_id in enumerate(moodle_id_list):
    print(ctg_id_list[i])
    
    link = "http://acpsem.moodle.com.au/mod/assign/view.php?id={}&action=view".format(moodle_id)
    result = session.get(link)
    
    m = re.search(submission_status_string, result.text)
    submission_status.append(m.group(2))
    print(submission_status[i])

    m = re.search(grading_status_string, result.text)
    grading_status.append(m.group(2))
    print(grading_status[i])

    if grading_status[i] == 'Graded':
        m = re.search(grade_string, result.text)
        grade.append(m.group(1))
    else:
        grade.append('0.00')
    
    print(grade[i])
    print(" ")
    
del session

1.1.1.1
Submitted for grading
Graded
1.00
 
1.1.1.2
Submitted for grading
Graded
1.00
 
1.1.1.3
No attempt
Not graded
0.00
 
1.1.1.4
No attempt
Not graded
0.00
 
1.1.2.1
No attempt
Not graded
0.00
 
1.2.1.1
Submitted for grading
Graded
1.00
 
1.2.1.2
Submitted for grading
Graded
1.00
 
1.2.1.3
Submitted for grading
Graded
1.00
 
1.2.1.4
Submitted for grading
Graded
1.00
 
1.2.2.1
No attempt
Not graded
0.00
 
1.3.1.1
No attempt
Not graded
0.00
 
2.1.1.1
Submitted for grading
Graded
1.00
 
2.1.1.2
Submitted for grading
Graded
1.00
 
2.1.1.3
Submitted for grading
Graded
1.00
 
2.1.1.4
Submitted for grading
Graded
1.00
 
2.1.2.1
No attempt
Not graded
0.00
 
2.1.2.2
No attempt
Not graded
0.00
 
2.1.3.1
No attempt
Not graded
0.00
 
2.2.1.1
Submitted for grading
Graded
1.00
 
2.2.1.2
Submitted for grading
Graded
1.00
 
2.2.2.1
No attempt
Not graded
0.00
 
2.2.2.2
No attempt
Not graded
0.00
 
2.2.2.3
No attempt
Not graded
0.00
 
2.2.3.1
No attempt
Not graded
0.00
 
2.3.1.1
Submitted for gradin

In [23]:
pd.set_option('display.max_rows', 500)

df = pd.DataFrame(
    [submission_status, grade]
)

df = df.transpose()
df.columns = ["Moodle submission", "Grade on Moodle"]
df.index = ctg_id_list
df

Moodle submission Grade on Moodle
1.1.1.1  Submitted for grading            1.00
1.1.1.2  Submitted for grading            1.00
1.1.1.3             No attempt            0.00
1.1.1.4             No attempt            0.00
1.1.2.1             No attempt            0.00
1.2.1.1  Submitted for grading            1.00
1.2.1.2  Submitted for grading            1.00
1.2.1.3  Submitted for grading            1.00
1.2.1.4  Submitted for grading            1.00
1.2.2.1             No attempt            0.00
1.3.1.1             No attempt            0.00
2.1.1.1  Submitted for grading            1.00
2.1.1.2  Submitted for grading            1.00
2.1.1.3  Submitted for grading            1.00
2.1.1.4  Submitted for grading            1.00
2.1.2.1             No attempt            0.00
2.1.2.2             No attempt            0.00
2.1.3.1             No attempt            0.00
2.2.1.1  Submitted for grading            1.00
2.2.1.2  Submitted for grading            1.00
2.2.2.1             No attempt            0.00
2.2.2.2             No attempt            0.00
2.2.2.3             No attempt            0.00
2.2.3.1             No attempt            0.00
2.3.1.1  Submitted for grading            1.00
2.3.1.2  Submitted for grading            1.00
2.3.1.3  Submitted for grading            1.00
2.3.2.1             No attempt            0.00
2.3.2.2             No attempt            0.00
2.3.2.3             No attempt            0.00
2.3.3.1             No attempt            0.00
2.4.1.1  Submitted for grading            1.00
2.4.1.2  Submitted for grading            1.00
2.4.1.3  Submitted for grading            1.00
2.4.1.4  Submitted for grading            1.00
2.4.2.1             No attempt            0.00
2.4.2.2             No attempt            0.00
2.4.3.1             No attempt            0.00
2.5.1.1  Submitted for grading            1.00
2.5.2.1             No attempt            0.00
2.5.2.2             No attempt            0.00
2.5.2.3             No attempt            0.00
2.5.3.1             No attempt            0.00
2.6.1.1  Submitted for grading            1.00
2.6.1.2  Submitted for grading            1.00
2.6.2.1             No attempt            0.00
2.6.2.2             No attempt            0.00
2.6.3.1             No attempt            0.00
3.1.1.1  Submitted for grading            1.00
3.1.1.2  Submitted for grading            1.00
3.1.1.3  Submitted for grading            1.00
3.1.1.4  Submitted for grading            1.00
3.1.2.1             No attempt            0.00
3.1.2.2  Submitted for grading            1.00
3.1.3.1             No attempt            0.00
3.2.1.1  Submitted for grading            1.00
3.2.2.1  Submitted for grading            1.00
3.2.2.2             No attempt            0.00
3.2.3.1             No attempt            0.00
3.3.1.1  Submitted for grading            1.00
3.3.1.2  Submitted for grading            1.00
3.3.1.3  Submitted for grading            1.00
3.3.2.1  Submitted for grading            0.70
3.3.2.2  Submitted for grading            0.70
3.3.2.3  Submitted for grading            0.70
3.3.2.4  Submitted for grading            0.70
3.3.3.1             No attempt            0.00
3.4.1.1  Submitted for grading            1.00
3.4.1.2  Submitted for grading            1.00
3.4.2.1  Submitted for grading            0.70
3.4.2.2  Submitted for grading            1.00
3.4.3.1             No attempt            0.00
3.5.1.1  Submitted for grading            1.00
3.5.1.2  Submitted for grading            1.00
3.5.2.1             No attempt            0.00
3.5.2.2             No attempt            0.00
3.5.3.1             No attempt            0.00
4.1.1.1  Submitted for grading            1.00
4.1.1.2  Submitted for grading            1.00
4.1.2.1  Submitted for grading            1.00
4.1.2.2  Submitted for grading            1.00
4.1.2.3  Submitted for grading            1.00
4.1.3.1             No attempt            0.00
4.2.1.1  Submitted for grading            1.00
4.2.1.2  Submitted

In [24]:
df.to_csv("MoodleGrades.csv")